In [14]:
import numpy as np
import time
import pickle
import os
import matplotlib.pyplot as plt
import warnings
import pandas as pd

warnings.filterwarnings('ignore')

In [21]:
print("----------------Reading the Data-------------------------")
PATH = os.getcwd()
os.chdir('Alphabets/')

X_train = pd.read_csv('train.csv', sep=',', header=None, index_col=False)
X_test = pd.read_csv('test.csv', sep=',', header=None, index_col=False)
np.random.shuffle(X_train.to_numpy())
train_class = X_train[X_train.columns[-1]]
test_actual_class = X_test[X_test.columns[-1]]

X_train = X_train.drop(X_train.columns[-1], axis=1)
X_test = X_test.drop(X_test.columns[-1], axis=1)

print("----------------Data Reading completed-------------------")

os.chdir('../')

X_train = X_train/255
X_test = X_test/255

m = X_train.shape[0] # Number of Training Samples

X_valid = X_train.iloc[(int(0.85*m)):]
valid_class = train_class[(int(0.85*m)):]
X_train = X_train.iloc[0:int(0.85*m)]
train_class = train_class[0:int(0.85*m)]


m = X_train.shape[0] # Number of Training Samples
n = X_train.shape[1] # Number of input features

print("The total number of training samples = {}".format(m))
print("The total number of validation samples = {}".format(X_valid.shape[0]))

print("The number of features = {}".format(n))

----------------Reading the Data-------------------------
----------------Data Reading completed-------------------
The total number of training samples = 11050
The total number of validation samples = 1950
The number of features = 784


In [22]:
#To get the one hot encoding of each label
print("--------Perform 1-hot encoding of class labels------------")

train_class_enc = pd.get_dummies(train_class).to_numpy()
valid_class_enc = pd.get_dummies(valid_class).to_numpy()
test_actual_class_enc = pd.get_dummies(test_actual_class).to_numpy()

--------Perform 1-hot encoding of class labels------------


In [23]:
#Add the intercept term to the data samples both in training and test dataset
X_train = np.hstack((np.ones((m,1)),X_train.to_numpy()))
X_valid = np.hstack((np.ones((X_valid.shape[0],1)), X_valid.to_numpy()))
X_test = np.hstack((np.ones((X_test.shape[0],1)),X_test.to_numpy()))

In [24]:
lr = 0.1
arch_test = [1,5,10,50,100]
arch = [arch_test[3]] #means one hidden layer with 2 perceptrons 
batch_size = 100 # Mini-Batch Size
r = np.max(train_class) + 1 # Default value of the number of classes = 26

In [25]:
#Mini-Batch formation
mini_batch = [(X_train[i:i+batch_size,:], train_class_enc[i:i+batch_size]) for i in range(0, m, batch_size)]
print("The number of mini-batches formed is = {}".format(len(mini_batch)))

The number of mini-batches formed is = 111


In [26]:
#Theta Initialization 
def theta_init(arch=[50]):
    theta = []
    for i in range(len(arch)+1):
        if i == 0:
            dim0=n+1
            dim1=arch[i]
        elif (i == len(arch)):
            dim0=arch[i-1]
            dim1 = r
        else:
            dim0=arch[i-1]
            dim1= arch[i]

        theta.append(2*np.random.random((dim0, dim1))-1)
        #theta.append(np.zeros((dim0, dim1)))
    return theta

In [27]:
def activation(x):
    return 1/(1+np.exp(-x))

In [28]:
def forward_prop(data, theta):
    fm = []
    fm.append(data)
    for l in range(len(theta)):
        fm.append(activation(np.dot(fm[l], theta[l])))
    return fm

In [29]:
def cost_total(X, theta, Y, m):
    fm = forward_prop(X, theta)
    cost = (1/(2*m))*np.sum((Y-fm[-1])**2)
    return cost

In [30]:
def calc_accuracy(data, theta, actual_class):
    pred_class = forward_prop(data, theta)
    test_pred_class = pred_class[-1]
    for i in range(len(test_pred_class)):
        test_pred_class[i][test_pred_class[i] == np.max(test_pred_class[i])] = 1
        test_pred_class[i][test_pred_class[i] != np.max(test_pred_class[i])] = 0


    test_acc = 0
    for i in range(len(actual_class)):
        if (np.array_equal(test_pred_class[i], actual_class[i])):
            test_acc+=1
    test_acc /= data.shape[0]

    #print("The Test Accuracy of the model = {}%".format(test_acc*100))
    return (test_acc*100)

In [34]:
epochs = []
train_accuracy = []
test_accuracy = []
valid_accuracy=[]
train_time = []

In [ ]:
lr0=0.5
arch_test=[50]
for i in range(len(arch_test)):
    theta = theta_init([arch_test[i]])
    fm = forward_prop(X_train, theta)
    ite=1
    epoch = 0
    start = time.time()
    cost_init = cost_total(X_valid, theta, valid_class_enc, m)

    while(True):
        count = 0

        print("Initial Cost on Val dataset for this epoch {} = {}".format(epoch, cost_init))

        for b in mini_batch:
            lr = lr0/ite
            X_b = b[0]
            Y_b = b[1]
            fm = forward_prop(X_b, theta)
            delta = [None]*len(fm)

            if (count % 60 == 0):
                print("Error on this batch = "+str(cost_total(X_b, theta, Y_b, batch_size)))
            #Backward Propagation

            for l in range(len(fm)-1, 0, -1):
                if (l == len(fm)-1):
                    delta[l] = ((1/batch_size)*(Y_b - fm[l])*fm[l]*(1-fm[l]))
                else:
                    delta[l]=(np.dot(delta[l+1], theta[l].T)*fm[l]*(1-fm[l]))

            #delta_l1 = np.dot(delta_l2, theta2.T)*l1*(1-l1)

            for t in range(len(theta)):
                theta[t] += lr*np.dot(fm[t].T, delta[t+1]) 

            count+=1
            ite+=1
        epoch+=1 #Number of epochs
        cost_final = cost_total(X_valid, theta, valid_class_enc, m)
        print("Cost on val dataset after {} epochs is = {}".format(epoch, cost_final))
        if (abs(cost_final-cost_init) < 1e-07):
            print("cost initial= {} , cost final={} , change in cost= {}".format(cost_init,cost_final, cost_final-cost_init))
            break
        cost_init = cost_final
    epochs.append(epoch)
    train_time.append(time.time()-start)
    train_accuracy.append(calc_accuracy(X_train, theta, train_class_enc))
    valid_accuracy.append(calc_accuracy(X_valid, theta, valid_class_enc))
    test_accuracy.append(calc_accuracy(X_test, theta, test_actual_class_enc))
    print("\n------------------------------------------------------------------------------")
    print("The stats for number of units in the hidden layer = {} are as below:".format(arch_test[i]))
    print("------------------------------------------------------------------------------")
    print("The number of epochs = {:2.3f}".format(epochs[-1]))
    print("The training time = {:2.3f}sec".format(train_time[-1]))
    print("The training accuracy is = {:2.3f}%".format(train_accuracy[-1]))
    print("The validation accuracy is = {:2.3f}%".format(valid_accuracy[-1]))
    print("The test accuracy is = {:2.3f}%".format(test_accuracy[-1]))
    print("------------------------------------------------------------------------------\n")

Initial Cost on Val dataset for this epoch 0 = 1.0440298228788043
Error on this batch = 5.812012260115962
Error on this batch = 1.7231636157464345
Cost on val dataset after 1 epochs is = 0.26760650714746553
Initial Cost on Val dataset for this epoch 1 = 0.26760650714746553
Error on this batch = 1.4598936398646547
Error on this batch = 1.3189732368206815
Cost on val dataset after 2 epochs is = 0.22394390326149904
Initial Cost on Val dataset for this epoch 2 = 0.22394390326149904
Error on this batch = 1.2195706836759337
Error on this batch = 1.1705900318929392
Cost on val dataset after 3 epochs is = 0.2034191293929951
Initial Cost on Val dataset for this epoch 3 = 0.2034191293929951
Error on this batch = 1.1071357450283423
Error on this batch = 1.087708932474216
Cost on val dataset after 4 epochs is = 0.19098213584204962
Initial Cost on Val dataset for this epoch 4 = 0.19098213584204962
Error on this batch = 1.0391934074552072
Error on this batch = 1.0331223025551568
Cost on val dataset 

Error on this batch = 0.7510389299526768
Cost on val dataset after 41 epochs is = 0.13494996483966787
Initial Cost on Val dataset for this epoch 41 = 0.13494996483966787
Error on this batch = 0.7364223643874942
Error on this batch = 0.7492405958384691
Cost on val dataset after 42 epochs is = 0.134632857847875
Initial Cost on Val dataset for this epoch 42 = 0.134632857847875
Error on this batch = 0.7347414723073719
Error on this batch = 0.7475033940076352
Cost on val dataset after 43 epochs is = 0.13432639119106937
Initial Cost on Val dataset for this epoch 43 = 0.13432639119106937
Error on this batch = 0.7331175957692605
Error on this batch = 0.7458238356422551
Cost on val dataset after 44 epochs is = 0.1340299637899541
Initial Cost on Val dataset for this epoch 44 = 0.1340299637899541
Error on this batch = 0.7315474940610028
Error on this batch = 0.7441987065888265
Cost on val dataset after 45 epochs is = 0.1337430214643988
Initial Cost on Val dataset for this epoch 45 = 0.13374302146

Cost on val dataset after 82 epochs is = 0.12700104080303024
Initial Cost on Val dataset for this epoch 82 = 0.12700104080303024
Error on this batch = 0.6944970629189277
Error on this batch = 0.7055313631558471
Cost on val dataset after 83 epochs is = 0.1268811377366924
Initial Cost on Val dataset for this epoch 83 = 0.1268811377366924
Error on this batch = 0.6938682450787381
Error on this batch = 0.7048701781627567
Cost on val dataset after 84 epochs is = 0.1267632356127127
Initial Cost on Val dataset for this epoch 84 = 0.1267632356127127
Error on this batch = 0.6932500325804185
Error on this batch = 0.7042199954139042
Cost on val dataset after 85 epochs is = 0.12664727694640454
Initial Cost on Val dataset for this epoch 85 = 0.12664727694640454
Error on this batch = 0.6926421187618333
Error on this batch = 0.7035805004193256
Cost on val dataset after 86 epochs is = 0.12653320657059255
Initial Cost on Val dataset for this epoch 86 = 0.12653320657059255
Error on this batch = 0.6920442

Error on this batch = 0.6853990516482635
Cost on val dataset after 123 epochs is = 0.12328268749314865
Initial Cost on Val dataset for this epoch 123 = 0.12328268749314865
Error on this batch = 0.6750513480902344
Error on this batch = 0.6850166121816466
Cost on val dataset after 124 epochs is = 0.12321416093497962
Initial Cost on Val dataset for this epoch 124 = 0.12321416093497962
Error on this batch = 0.6746940696138227
Error on this batch = 0.6846384288173806
Cost on val dataset after 125 epochs is = 0.12314639078804404
Initial Cost on Val dataset for this epoch 125 = 0.12314639078804404
Error on this batch = 0.6743407748184884
Error on this batch = 0.6842644194133686
Cost on val dataset after 126 epochs is = 0.12307936253846591
Initial Cost on Val dataset for this epoch 126 = 0.12307936253846591
Error on this batch = 0.6739913867783947
Error on this batch = 0.6838945040642946
Cost on val dataset after 127 epochs is = 0.12301306206569859
Initial Cost on Val dataset for this epoch 12

Error on this batch = 0.6725132498697051
Cost on val dataset after 164 epochs is = 0.12097029134459635
Initial Cost on Val dataset for this epoch 164 = 0.12097029134459635
Error on this batch = 0.6630179356285685
Error on this batch = 0.6722557891671348
Cost on val dataset after 165 epochs is = 0.12092401594340031
Initial Cost on Val dataset for this epoch 165 = 0.12092401594340031
Error on this batch = 0.6627776107541291
Error on this batch = 0.6720004616111931
Cost on val dataset after 166 epochs is = 0.12087812110496936
Initial Cost on Val dataset for this epoch 166 = 0.12087812110496936
Error on this batch = 0.662539281413635
Error on this batch = 0.6717472363695752
Cost on val dataset after 167 epochs is = 0.12083260135714055
Initial Cost on Val dataset for this epoch 167 = 0.12083260135714055
Error on this batch = 0.6623029187353648
Error on this batch = 0.6714960832410969
Cost on val dataset after 168 epochs is = 0.12078745133929203
Initial Cost on Val dataset for this epoch 168

...

Error on this batch = 0.5956652922742124
Cost on val dataset after 2670 epochs is = 0.1070231964799416
Initial Cost on Val dataset for this epoch 2670 = 0.1070231964799416
Error on this batch = 0.5915127981208654
Error on this batch = 0.5956586915762694
Cost on val dataset after 2671 epochs is = 0.10702198582361755
Initial Cost on Val dataset for this epoch 2671 = 0.10702198582361755
Error on this batch = 0.59150667330538
Error on this batch = 0.5956520940078325
Cost on val dataset after 2672 epochs is = 0.10702077573878979
Initial Cost on Val dataset for this epoch 2672 = 0.10702077573878979
Error on this batch = 0.5915005513948184
Error on this batch = 0.5956454995662217
Cost on val dataset after 2673 epochs is = 0.10701956622497026
Initial Cost on Val dataset for this epoch 2673 = 0.10701956622497026
Error on this batch = 0.5914944323866912
Error on this batch = 0.5956389082487605
Cost on val dataset after 2674 epochs is = 0.10701835728167147
Initial Cost on Val dataset for this epo

Cost on val dataset after 2708 epochs is = 0.1069775892276845
Initial Cost on Val dataset for this epoch 2708 = 0.1069775892276845
Error on this batch = 0.5912820765191485
Error on this batch = 0.5954101597955114
Cost on val dataset after 2709 epochs is = 0.10697639995007664
Initial Cost on Val dataset for this epoch 2709 = 0.10697639995007664
Error on this batch = 0.5912760603664142
Error on this batch = 0.5954036791922718
Cost on val dataset after 2710 epochs is = 0.1069752112258315
Initial Cost on Val dataset for this epoch 2710 = 0.1069752112258315
Error on this batch = 0.5912700470261337
Error on this batch = 0.5953972016164245
Cost on val dataset after 2711 epochs is = 0.10697402305448309
Initial Cost on Val dataset for this epoch 2711 = 0.10697402305448309
Error on this batch = 0.5912640364959313
Error on this batch = 0.5953907270654148
Cost on val dataset after 2712 epochs is = 0.10697283543556606
Initial Cost on Val dataset for this epoch 2712 = 0.10697283543556606
Error on th

Cost on val dataset after 2749 epochs is = 0.10692927769312228
Initial Cost on Val dataset for this epoch 2749 = 0.10692927769312228
Error on this batch = 0.5910376954333939
Error on this batch = 0.5951469106023299
Cost on val dataset after 2750 epochs is = 0.10692811072727602
Initial Cost on Val dataset for this epoch 2750 = 0.10692811072727602
Error on this batch = 0.5910317926735489
Error on this batch = 0.5951405520599604
Cost on val dataset after 2751 epochs is = 0.10692694429614462
Initial Cost on Val dataset for this epoch 2751 = 0.10692694429614462
Error on this batch = 0.5910258926310777
Error on this batch = 0.595134196442738
Cost on val dataset after 2752 epochs is = 0.10692577839928472
Initial Cost on Val dataset for this epoch 2752 = 0.10692577839928472
Error on this batch = 0.5910199953037195
Error on this batch = 0.5951278437482315
Cost on val dataset after 2753 epochs is = 0.10692461303625336
Initial Cost on Val dataset for this epoch 2753 = 0.10692461303625336
Error on

Cost on val dataset after 2789 epochs is = 0.10688301180385103
Initial Cost on Val dataset for this epoch 2789 = 0.10688301180385103
Error on this batch = 0.5908036825001368
Error on this batch = 0.5948948267656953
Cost on val dataset after 2790 epochs is = 0.10688186588590137
Initial Cost on Val dataset for this epoch 2790 = 0.10688186588590137
Error on this batch = 0.5907978866982849
Error on this batch = 0.5948885833612826
Cost on val dataset after 2791 epochs is = 0.10688072048533578
Initial Cost on Val dataset for this epoch 2791 = 0.10688072048533578
Error on this batch = 0.5907920935254523
Error on this batch = 0.5948823427869995
Cost on val dataset after 2792 epochs is = 0.10687957560173135
Initial Cost on Val dataset for this epoch 2792 = 0.10687957560173135
Error on this batch = 0.5907863029794832
Error on this batch = 0.5948761050405277
Cost on val dataset after 2793 epochs is = 0.10687843123466585
Initial Cost on Val dataset for this epoch 2793 = 0.10687843123466585
Error o

Cost on val dataset after 2829 epochs is = 0.10683757451304504
Initial Cost on Val dataset for this epoch 2829 = 0.10683757451304504
Error on this batch = 0.5905738799956638
Error on this batch = 0.5946472754355836
Cost on val dataset after 2830 epochs is = 0.10683644896551697
Initial Cost on Val dataset for this epoch 2830 = 0.10683644896551697
Error on this batch = 0.5905681876981664
Error on this batch = 0.5946411434549626
Cost on val dataset after 2831 epochs is = 0.10683532391884182
Initial Cost on Val dataset for this epoch 2831 = 0.10683532391884182
Error on this batch = 0.5905624979454211
Error on this batch = 0.5946350142138449
Cost on val dataset after 2832 epochs is = 0.10683419937261607
Initial Cost on Val dataset for this epoch 2832 = 0.10683419937261607
Error on this batch = 0.5905568107353709
Error on this batch = 0.5946288877100189
Cost on val dataset after 2833 epochs is = 0.10683307532643681
Initial Cost on Val dataset for this epoch 2833 = 0.10683307532643681
Error o

In [43]:
calc_accuracy(X_test, theta, test_actual_class_enc)

84.06153846153846

In [44]:
train_accuracy

[94.95927601809954, 93.78280542986425, 92.85972850678733]